# Исходные данные

In [1]:
import sympy
import IPython.display as disp
from iapws import IAPWS97
import pandas as pd
import math
import random
import pyvista
import numpy
from ansys.mapdl.core import launch_mapdl
def fr(st,pos):
    if pos==True:
        return sympy.symbols(st, real=True, positive=True) 
    else:
        return sympy.symbols(st, real=True) 
Q_p=fr("Q_p",True)#МВт
q_v=fr("q_v",True)#МВт/м^2
T_p=fr("T_p",True)#компания реактора в часах
P_1=fr("P_1",True)
P_1f=16#МПа
t_aver=fr("t_aver",True)
t_averf=307+273#K
#тип ТВС-2М
t_U=fr("t_U",True)
t_Uf=890+273.15
d_FE_out=fr("d_FE_out",True)
d_FE_outf=9.1/1000
d_f=fr("d_f",True)
d_ff=7.6/1000
S_FE=fr("S_FE",True)
S_FEf=12.75/1000
H_frac_D=fr("H_frac_D",True)
H_frac_Df=1
h_k=fr("h_k",True)
h_kf=235/1000
n_p=fr("n_p",True)
n_pf=18
d_p=fr("d_p",True)
d_pf=13/1000
b_p=fr("b_p",True)
b_pf=1/1000
d_c=fr("d_c",True)
d_cf=10.3/1000
b_c=fr("b_c",True)
b_cf=0.65/1000
nf=fr("nf",True)
nff=4
d_FE_in=fr("d_FE_in",True)
d_FE_inf=7.73/1000
m=fr("m",True)
mf=21
#______________________________________________________________________

b_ceil=fr("𝛿_ceil",True)
b_ceilf=2/1000#межкассетный зазор
#чехла нет

In [2]:
import numpy as np
df=pd.read_excel('G.xlsx',index_col=[0],parse_dates=[0])

## Расчет геометрических характеристик

In [3]:
data=pd.DataFrame(columns=["Name","Num"])
pi=sympy.pi
graph={Q_p: 1399,q_v:83,T_p:7000,P_1:P_1f,t_aver:t_averf,t_U:t_Uf,d_FE_out:d_FE_outf,
                        d_f:d_ff,S_FE:S_FEf,H_frac_D:H_frac_Df,h_k:h_kf,n_p:n_pf,d_p:d_pf,b_p:b_pf,
                       d_c:d_cf,b_c:b_cf,nf:nff,d_FE_in:d_FE_inf,b_ceil:b_ceilf,m:mf}
data=pd.DataFrame(columns=["Name","Num"])
def find_dens(t):
    s=0
    for i in range(len(denstab)):
        s+=denstab[i]*t**i
    return s
for i in graph:
    data=pd.concat([data,pd.DataFrame([[i,graph[i]]],columns=["Name","Num"])])
def Pow(a,b,ev=False):
    return sympy.Pow(a,sympy.simplify(b),evaluate=ev)
transfer_data=True
transfer_data2=True
Showing=True
def constant(n):
    return sympy.simplify(str(n))
def add_in_table(st,num,calc=True,QU=3,show=True):
    global data
    xxx=sympy.symbols(st, real=True, positive=True) 
    eq=sympy.Eq(xxx,num,evaluate=False)
    if show==True and Showing==True:
        disp.display(eq)
    #print(graph)
    num2=num
    try:
        num=num.evalf(subs=graph)
    except Exception:
        num=constant(num)
    graph.update({xxx:num})
    if transfer_data==True:
        data=pd.concat([data,pd.DataFrame([[st,num]],columns=["Name","Num"])])
    if calc==True and Showing==True:
        eq=sympy.Eq(xxx,sympy.N(num,QU))
        disp.display(eq)
    if transfer_data2==True:
        return xxx
    else:
        return num2
def line(n):
    for i in range(n):
        print("________________________________________________________________________________________________________")
        

In [4]:
transfer_data=True
transfer_data2=True
V=add_in_table("V",Q_p/q_v)
#print(graph)
D=add_in_table("D",Pow((4*V/pi),"1/3"),QU=4)
F_k=add_in_table("F_k",sympy.sqrt(3)/2*Pow((h_k+b_ceil),"2"))
N_ceil=add_in_table("N_ceil_f",sympy.pi*Pow(D,"2")/4/F_k)
disp.display(disp.Latex("Из предположения:"))
n_cas=sympy.symbols("n_cas", real=True, positive=True)
n=add_in_table("n",sympy.solvers.solve(sympy.sqrt((n_cas*(h_k+b_ceil)+h_k/2)**2+(h_k/2/sympy.sqrt(3))**2)-D/2,n_cas,evaluate=False)[1])#.evalf(subs=graph)
graph[n]=math.floor(graph[n])
disp.display(sympy.Eq(n,graph[n]))
#Есть формула под другой вариант геометрии см. листинг расчетов МРИК
n_casset=add_in_table("n_ceil",3*n**2+3*n+1)
disp.display(disp.Latex("Из Эпюры:"))
graph[n_casset]=109
disp.display(sympy.Eq(n_casset,graph[n_casset]))
K=add_in_table("K",n_casset/N_ceil)
n_FE=add_in_table("n_FE",sympy.Rational(3,4)*(m**2-1)-n_p)
F_ceil=add_in_table("F_ceil",sympy.sqrt(3)/2*S_FE**2)
F_fuel=add_in_table("F_fuel",d_f**2/4*sympy.pi)
F_shell=add_in_table("F_shell",sympy.pi/4*(-d_FE_in**2+d_FE_out**2))
F_HE=add_in_table("F_HE",sympy.pi/4*(-d_f**2+d_FE_in**2))
F_moder=add_in_table("F_moder",F_ceil-F_fuel-F_shell-F_HE)
W=add_in_table("W",F_moder/F_fuel)
n_rod=add_in_table("n_rod",(m**2*3+1)/4)
a=add_in_table("a",sympy.Rational(1,1000),calc=False,show=False)
F_MK=add_in_table("F_MK",sympy.sqrt(3)/2*((h_k+b_ceil)**2-(h_k)**2)+a*sympy.sqrt(3)/2*(-(h_k-2*b_ceil)**2+(h_k)**2)*n_FE)
f_cm=add_in_table("f_cm",F_shell*n_FE+pi/4*(d_c**2-(d_c-2*b_c)**2)+pi/4*(d_p**2-(d_p-2*b_p)**2)*n_p)
f_cmE=add_in_table("f_cmE",f_cm/n_FE)
f_moderE=add_in_table("f_moderE",(sympy.sqrt(3)/2*h_k**2-n_FE*sympy.pi/4*d_FE_out**2-sympy.pi/4*d_c**2-n_p*sympy.pi/4*d_p**2+F_MK)/n_FE)
f_1=add_in_table("f_1",f_cmE+F_HE+f_moderE)
f_E=add_in_table('f_E',F_fuel+f_1)
R_E=add_in_table('R_E',sympy.sqrt(f_E/sympy.pi))



Eq(V, Q_p/q_v)

Eq(V, 16.9)

Eq(D, (4*V/pi)**(1/3))

Eq(D, 2.779)

Eq(F_k, sqrt(3)*(h_k + 𝛿_ceil)**2/2)

...

Eq(f_1, 0.000106)

Eq(f_E, F_fuel + f_1)

Eq(f_E, 0.000152)

Eq(R_E, sqrt(f_E)/sqrt(pi))

Eq(R_E, 0.00695)

In [5]:
transfer_data=True
transfer_data2=True
N_a= add_in_table("N_a",sympy.Rational('6.022140857') * Pow(10,"23",ev=True),calc=False,show=False)
#
graph_copy=graph.copy()
transfer_data=True
transfer_data2=True
Showing=True
def main(xi=None,Temp=False,TTT=t_averf,VAR=None,cold=True,mas=None):
    global graph, graph_copy
    graph=graph_copy.copy()
    x=sympy.symbols("X", real=True, positive=True)
    if xi!=None:
        if Showing==True:
            eq=sympy.Eq(x,xi)
            disp.display(eq)
        graph.update({x:xi})
    #sigma_a
    sigma_a5_0=add_in_table("σ_a5_0",Pow(10,"-24")*680.9,calc=False,show=False)
    sigma_a8_0=add_in_table("σ_a8_0",Pow(10,"-24")*2.71,calc=False,show=False)
    sigma_aZr_0=add_in_table("σ_aZr_0",Pow(10,"-24")*0.185,calc=False,show=False)
    sigma_aO2_0=add_in_table("σ_aO2_0",Pow(10,"-24")*0.00027,calc=False,show=False)
    sigma_ass_0=add_in_table("σ_ass_0",Pow(10,"-24")*2.87,calc=False,show=False)
    sigma_aHe_0=add_in_table("σ_aHe_0",Pow(10,"-24")*0.007,calc=False,show=False)
    sigma_aH2O_0=add_in_table("σ_aH2O_0",Pow(10,"-24")*0.66,calc=False,show=False)
    
    sigma_s5=add_in_table("σ_s5",Pow(10,"-24")*13.8,calc=False)
    sigma_s8=add_in_table("σ_s8",Pow(10,"-24")*8.9,calc=False)
    sigma_sZr=add_in_table("σ_sZr",Pow(10,"-24")*6.4,calc=False)
    sigma_sO2=add_in_table("σ_sO2",Pow(10,"-24")*3.76,calc=False)
    sigma_sss=add_in_table("σ_sss",Pow(10,"-24")*11.0,calc=False)
    sigma_sHe=add_in_table("σ_sHe",Pow(10,"-24")*0.8,calc=False)
    sigma_sH2O=add_in_table("σ_sH2O",Pow(10,"-24")*103.0,calc=False)

    sigma_f5_0=add_in_table("σ_f5_0",Pow(10,"-24")*583.5,calc=False,show=False)
    
    T_0=add_in_table("T_0",constant(293.2),calc=False,show=False)
    if Temp==False:
        disp.display(disp.Latex("Для холодного реактора:"))
        disp.display(sympy.Eq(T_0,graph[T_0]))
        sigma_a5=add_in_table("σ_a5",sigma_a5_0,show=False)
        sigma_a8=add_in_table("σ_a8",sigma_a8_0,show=False)
        sigma_aZr=add_in_table("σ_aZr",sigma_aZr_0,show=False)
        sigma_aO2=add_in_table("σ_aO2",sigma_aO2_0,show=False)
        sigma_ass=add_in_table("σ_ass",sigma_ass_0,show=False)
        sigma_aHe=add_in_table("σ_aHe",sigma_aHe_0,show=False)
        sigma_aH2O=add_in_table("σ_aH2O",sigma_aH2O_0,show=False)
        dens_H2O=add_in_table("ρ_H2O",constant(1),calc=False)
        sigma_f5=add_in_table("σ_f5",sigma_f5_0,show=False)
        
        Sigma_ksi_sH2O=add_in_table("Σξ_sH2O",1.35,calc=False)
        Sigma_ksi_sHe=add_in_table("Σξ_sHe",9.11e-6,calc=False)
        Sigma_ksi_scm=add_in_table("Σξ_scm",0.0283,calc=False)
        T=add_in_table("T",T_0,show=False,calc=False)
    elif Temp==True:
        if VAR!=-1:
            T=add_in_table("T",TTT,calc=False)
        if VAR==-1:
            T=sympy.symbols("T")
        if cold==True:
            dens_H2O=add_in_table("ρ_H2O",constant(1),calc=False)
        else:
            try:
                dens_H2O=add_in_table("ρ_H2O",constant(IAPWS97(P=P_1f,T=t_averf).rho)/1000,calc=False)
            except Exception:
                dens_H2O=add_in_table("ρ_H2O",find_dens(T))
    
        def ggg(a=True,t=None,U=True):
            try:
                tt=t.evalf(subs=graph)
            except Exception:
                tt=t
            df2=pd.DataFrame([[np.nan,np.nan,np.nan,np.nan]],columns=["ga5","gf5","ga9","gf9"],index=[tt])
            df2.index.name="T"
            df3=df.append(df2).sort_index().interpolate()
            #print(df3)
            if a==True:
                if U==True:
                    return df3.loc[tt,"ga5"]
                else:
                    return df3.loc[tt,"ga9"]
            else:
                if U==True:
                    return df3.loc[tt,"gf5"]
                else:
                    return df3.loc[tt,"gf9"]
        g_a_5=add_in_table("g_a_5",ggg(a=True,t=TTT),show=False)#тут 
        g_f_5=add_in_table("g_f_5",ggg(a=False,t=TTT),show=False)#тут
        sigma_a5=add_in_table("σ_a5",g_a_5*sympy.sqrt(sympy.pi)/2*sympy.sqrt(T_0/T)*sigma_a5_0)
        sigma_a8=add_in_table("σ_a8",sympy.sqrt(sympy.pi)/2*sympy.sqrt(T_0/T)*sigma_a8_0)
        sigma_aZr=add_in_table("σ_aZr",sympy.sqrt(sympy.pi)/2*sympy.sqrt(T_0/T)*sigma_aZr_0)
        sigma_aO2=add_in_table("σ_aO2",sympy.sqrt(sympy.pi)/2*sympy.sqrt(T_0/T)*sigma_aO2_0)
        sigma_aHe=add_in_table("σ_aHe",sympy.sqrt(sympy.pi)/2*sympy.sqrt(T_0/T)*sigma_aHe_0)
        sigma_aH2O=add_in_table("σ_aH2O",sympy.sqrt(sympy.pi)/2*sympy.sqrt(T_0/T)*sigma_aH2O_0)
        sigma_f5=add_in_table("σ_f5",g_f_5*sympy.sqrt(sympy.pi)/2*sympy.sqrt(T_0/T)*sigma_f5_0)
        sigma_ass=add_in_table("σ_ass",sympy.sqrt(sympy.pi)/2*sympy.sqrt(T_0/T)*sigma_ass_0)
        
        Sigma_ksi_sH2O=add_in_table("Σξ_sH2O",1.35*dens_H2O,show=False)
        Sigma_ksi_sHe=add_in_table("Σξ_sHe",9.11e-6,calc=False)
        Sigma_ksi_scm=add_in_table("Σξ_scm",0.0283,calc=False)
    A_H2O=add_in_table("A_H2O",constant(18),calc=False)
    N_H2O=add_in_table("N_H2O",N_a*dens_H2O/A_H2O)
    
    dens_UO2=add_in_table("ρ_UO2",constant(10),calc=False)
    A_UO2=add_in_table("A_UO2",constant(270.7),calc=False)
    N_UO2=add_in_table("N_UO2",N_a*dens_UO2/A_UO2)
    N_U=add_in_table("N_U",N_UO2)
    N_O=add_in_table("N_O",2*N_UO2)
    N_5=add_in_table("N_5",x*N_U)
    N_8=add_in_table("N_8",(1-x)*N_U)
    N_ss=add_in_table("N_ss",0.08978*1e24,calc=False)
    
    dens_Zr=add_in_table("ρ_Zr",constant(6.4),calc=False)
    A_Zr=add_in_table("A_Zr",constant(91.22),calc=False)
    N_Zr=add_in_table("N_Zr",N_a*dens_Zr/A_Zr)
    
    dens_He=add_in_table("ρ_He",Pow(10,"-5")*17.8,calc=False)
    A_He=add_in_table("A_He",constant(4.003),calc=False)
    N_He=add_in_table("N_He",N_a*dens_He/A_He)
    
    H=add_in_table("H",D)
    m_UO2=add_in_table("m_UO2",F_fuel*H*n_FE*n_casset*dens_UO2*1000)
    A_8=add_in_table("A_8",constant(238),calc=False)
    A_5=add_in_table("A_5",constant(235),calc=False)
    m_5=add_in_table("m_5",m_UO2*A_8/A_UO2/(1+(1-x)/x*A_8/A_5))
    Sigma_a5=add_in_table("Σ_a5",N_5*sigma_a5)
    Sigma_s5=add_in_table("Σ_s5",N_5*sigma_s5)
    Sigma_f5=add_in_table("Σ_f5",N_5*sigma_f5)
    
    Sigma_a8=add_in_table("Σ_a8",N_8*sigma_a8)
    Sigma_s8=add_in_table("Σ_s8",N_8*sigma_s8)
    
    Sigma_aO2=add_in_table("Σ_aO2",N_O*sigma_aO2)
    Sigma_sO2=add_in_table("Σ_sO2",N_O*sigma_sO2)
    
    Sigma_acm=add_in_table("Σ_acm",0.2*N_Zr*sigma_aZr+0.8*N_ss*sigma_ass,show=False)
    Sigma_scm=add_in_table("Σ_scm",0.2*N_Zr*sigma_sZr+0.8*N_ss*sigma_sss,show=False)
    
    Sigma_aHe=add_in_table("Σ_aHe",N_He*sigma_aHe)
    Sigma_sHe=add_in_table("Σ_sHe",N_He*sigma_sHe)
    
    Sigma_aH2O=add_in_table("Σ_aH2O",N_H2O*sigma_aH2O)
    Sigma_sH2O=add_in_table("Σ_sH2O",N_H2O*sigma_sH2O)
    
    Sigma_a_0=add_in_table("Sigma_a_0",Sigma_a5+Sigma_a8+Sigma_aO2)
    Sigma_s_0=add_in_table("Sigma_s_0",Sigma_s5+Sigma_s8+Sigma_sO2)
    
    Sigma_a_1=add_in_table("Sigma_a_1",(Sigma_acm*f_cmE+F_HE*Sigma_aHe+f_moderE*Sigma_aH2O)/(f_1))
    Sigma_s_1=add_in_table("Sigma_s_1",(Sigma_scm*f_cmE+F_HE*Sigma_sHe+f_moderE*Sigma_sH2O)/(f_1))
    
    Sigma_ksi_s_1=add_in_table("Σξ_s_1",(Sigma_ksi_scm*f_cmE+F_HE*Sigma_ksi_sHe+f_moderE*Sigma_ksi_sH2O)/(f_1))
    if VAR==None or VAR==-1:
        if cold==False:
            T_H=add_in_table("T_H",t_aver*(1+1.7*(Sigma_a_0*F_fuel+Sigma_a_1*f_1)/(Sigma_ksi_s_1*f_1)))#K
        elif cold==True:
            T_H=add_in_table("T_H",T_0*(1+1.7*(Sigma_a_0*F_fuel+Sigma_a_1*f_1)/(Sigma_ksi_s_1*f_1)))#K
        if VAR==-1:
            return sympy.re(sympy.solvers.solve(T_H.evalf(subs=graph)-T,T)[0])
    if VAR==1 or VAR==44:
        omega5=add_in_table("omega_5",2/3/A_5,show=True)
        Sigma_trs5=add_in_table("Sigma_trs5",Sigma_s5*(1-omega5))
        Sigma_tr5=add_in_table("Sigma_tr5",Sigma_trs5+Sigma_a5)
        
        omega8=add_in_table("omega_8",2/3/A_8,show=True)
        Sigma_trs8=add_in_table("Sigma_trs8",Sigma_s8*(1-omega8))
        Sigma_tr8=add_in_table("Sigma_tr8",Sigma_trs8+Sigma_a8)
        
        omegaO2=add_in_table("omega_O2",2/3/32,show=False)
        Sigma_trsO2=add_in_table("Sigma_trsO2",Sigma_sO2*(1-omegaO2))
        Sigma_trO2=add_in_table("Sigma_trO2",Sigma_trsO2+Sigma_aO2)
        
        omegacm=add_in_table("omega_8",2/3/62.50446506,show=True)#ПРОВЕРИТЬ
        Sigma_trscm=add_in_table("Sigma_trscm",Sigma_scm*(1-omegacm))
        Sigma_trcm=add_in_table("Sigma_trcm",Sigma_trscm+Sigma_acm)
        
        omegaHe=add_in_table("omega_He",2/3/A_He,show=True)
        Sigma_trsHe=add_in_table("Sigma_trsHe",Sigma_sHe*(1-omegaHe))
        Sigma_trHe=add_in_table("Sigma_trHe",Sigma_trsHe+Sigma_aHe)
        Sigma_trH2O=add_in_table("Sigma_trH2O",(dens_H2O/2.72)**2*293/T*sympy.sqrt(1+0.0614*T/293)/1.0614*1/3/Sigma_aH2O)
        
        Sigma_tr0=add_in_table("Sigma_tr0",Sigma_tr5+Sigma_tr8+Sigma_trO2)
        Sigma_tr1=add_in_table("Sigma_tr1",(Sigma_trcm*f_cmE+F_HE*Sigma_trHe+f_moderE*Sigma_trH2O)/(f_cmE+F_HE+f_moderE))
        nu_f5=add_in_table("nu_f5",2.416,calc=False)
    if VAR==1 or VAR==44:
        eta=add_in_table("eta",nu_f5*Sigma_f5/Sigma_a_0)
    if VAR==2 or VAR==3:
        nu_f5=add_in_table("nu_f5",2.416,calc=False)
        E_f=add_in_table("E_f",3.2*1e-17)
        V_fuel=add_in_table("V_fuel",F_fuel/f_E*V)
        T_k=add_in_table("T_k",nf*T_p*3600)
        Fi=add_in_table("Phi",Q_p*1e-6/sigma_f5/N_5/E_f/V_fuel)
        z=add_in_table("z",Fi*T_k*sigma_f5)
        
        N_5end=add_in_table("N_5end",N_5*sympy.exp(-z))
        sigma_a9_0=add_in_table("sigma_a9_0",1011.2*1e-24,calc=False)
        sigma_f9_0=add_in_table("sigma_f9_0",744.07*1e-24,calc=False)
        sigma_s9=add_in_table("sigma_s9",7.7*1e-24,calc=False)
        g_a_9=add_in_table("g_a_9",ggg(a=True,t=TTT,U=False),show=False)#тут 
        g_f_9=add_in_table("g_f_9",ggg(a=False,t=TTT,U=False),show=False)
        sigma_a9=add_in_table("σ_a9",g_a_9*sympy.sqrt(sympy.pi)/2*sympy.sqrt(T_0/T)*sigma_a9_0)
        sigma_f9=add_in_table("σ_f9",g_f_9*sympy.sqrt(sympy.pi)/2*sympy.sqrt(T_0/T)*sigma_f9_0)
        
        epsilon_start=add_in_table("epsilon_start",mas[0],show=False)
        varphi_start=add_in_table("varphi_start",mas[1],show=False)
        nu_f9=add_in_table("nu_f9",2.862,show=False)
        aaa=add_in_table("aaa",sigma_a9/sigma_a5*(1-epsilon_start*(1-varphi_start)*nu_f9*sigma_f9/sigma_a9))
        N_9=add_in_table("N_9",N_8*sigma_a8/sigma_a5/aaa*(1-sympy.exp(-aaa*z))+nu_f5*sigma_f5*epsilon_start*(1-varphi_start)*N_5/sigma_a5/(aaa-1)*(sympy.exp(-z)-sympy.exp(-aaa*z)))
        N_slag=add_in_table("N_slag",z*(N_5end+N_9*sigma_f9/sigma_a5))
        C_slag=add_in_table("C_slag",N_slag/N_5)
        sigma_aslag=add_in_table("sigma_aslag",(85.65-112*C_slag+1.52*C_slag**2)*1e-24*sympy.Heaviside(C_slag.evalf(subs=graph),H0=1)-(85.65-112*C_slag+1.52*C_slag**2)*1e-24*sympy.Heaviside(C_slag.evalf(subs=graph)-0.4,H0=1)+(50-22.4*C_slag)*1e-24*sympy.Heaviside(C_slag.evalf(subs=graph)-0.4,H0=1)-(50-22.4*C_slag)*1e-24*sympy.Heaviside(C_slag.evalf(subs=graph)-0.8,H0=1)+(47-6.25*C_slag-12.5*C_slag**2)*1e-24*sympy.Heaviside(C_slag.evalf(subs=graph)-0.8,H0=1)-(47-6.25*C_slag-12.5*C_slag**2)*1e-24*sympy.Heaviside(C_slag.evalf(subs=graph)-1,H0=0))
        sigma_aXe_0=add_in_table("sigma_aXe_0",2650000*1e-24)
        sigma_aXe=add_in_table("sigma_aXe",sympy.sqrt(sympy.pi)/2*sigma_aXe_0*sympy.sqrt(T_0/T))
        gamma_Xe=add_in_table("gamma_Xe",0.003,show=False)
        gamma_I=add_in_table("gamma_I",0.06,show=False)
        lambda_Xe=add_in_table("lambda_Xe",2.1*1e-5,show=False)
        N_Xe=add_in_table("N_Xe",(gamma_Xe+gamma_I)*sigma_f5*N_5end*Fi/(lambda_Xe+sigma_aXe*Fi))
        
        sigma_aSm_0=add_in_table("sigma_aSm_0",41000*1e-24)
        sigma_aSm=add_in_table("sigma_aSm",sympy.sqrt(sympy.pi)/2*sigma_aSm_0*sympy.sqrt(T_0/T))
        gamma_Sm=add_in_table("gamma_Sm",0.013,show=False)
        
        N_Sm=add_in_table("N_Sm",gamma_Sm*sigma_f5*N_5end/sigma_aSm)
        del graph[Sigma_a5]
        Sigma_a5=add_in_table("Σ_a5",N_5end*sigma_a5)

        del graph[Sigma_f5]
        Sigma_f5=add_in_table("Σ_f5",N_5end*sigma_f5)
        Sigma_a9=add_in_table("Σ_a9",N_9*sigma_a9)
        Sigma_s9=add_in_table("Σ_s9",N_9*sigma_s9)
        Sigma_f9=add_in_table("Σ_f9",N_9*sigma_f9)
        Sigma_aslag=add_in_table("Σ_aslag",N_slag*sigma_aslag)
        Sigma_aXe=add_in_table("Σ_aXe",N_Xe*sigma_aXe)
        Sigma_aSm=add_in_table("Σ_aSm",N_Sm*sigma_aSm)
        del graph[Sigma_a_0]
        Sigma_a_0=add_in_table("Sigma_a_0",Sigma_a5+Sigma_a8+Sigma_aO2+Sigma_a9+Sigma_aslag+Sigma_aXe+Sigma_aSm)
        
        omega5=add_in_table("omega_5",2/3/A_5,show=True)
        Sigma_trs5=add_in_table("Sigma_trs5",Sigma_s5*(1-omega5))
        Sigma_tr5=add_in_table("Sigma_tr5",Sigma_trs5+Sigma_a5)
        
        omega8=add_in_table("omega_8",2/3/A_8,show=True)
        Sigma_trs8=add_in_table("Sigma_trs8",Sigma_s8*(1-omega8))
        Sigma_tr8=add_in_table("Sigma_tr8",Sigma_trs8+Sigma_a8)
        
        omegaO2=add_in_table("omega_O2",2/3/32,show=False)
        Sigma_trsO2=add_in_table("Sigma_trsO2",Sigma_sO2*(1-omegaO2))
        Sigma_trO2=add_in_table("Sigma_trO2",Sigma_trsO2+Sigma_aO2)
        
        omegacm=add_in_table("omega_8",2/3/62.50446506,show=True)#ПРОВЕРИТЬ
        Sigma_trscm=add_in_table("Sigma_trscm",Sigma_scm*(1-omegacm))#60.644
        Sigma_trcm=add_in_table("Sigma_trcm",Sigma_trscm+Sigma_acm)
        
        omegaHe=add_in_table("omega_He",2/3/A_He,show=True)
        Sigma_trsHe=add_in_table("Sigma_trsHe",Sigma_sHe*(1-omegaHe))
        Sigma_trHe=add_in_table("Sigma_trHe",Sigma_trsHe+Sigma_aHe)
        
        A_9=add_in_table("A_9",239)
        omega9=add_in_table("omega_9",2/3/A_9,show=True)
        Sigma_trs9=add_in_table("Sigma_trs9",Sigma_s9*(1-omega9))
        Sigma_tr9=add_in_table("Sigma_tr9",Sigma_trs9+Sigma_a9)        
        
        Sigma_trH2O=add_in_table("Sigma_trH2O",(dens_H2O/2.72)**2*293/T*sympy.sqrt(1+0.0614*T/293)/1.0614*1/3/Sigma_aH2O)
        
        Sigma_tr0=add_in_table("Sigma_tr0",Sigma_tr5+Sigma_tr8+Sigma_trO2+Sigma_tr9+Sigma_aslag+Sigma_aXe+Sigma_aSm)
        Sigma_tr1=add_in_table("Sigma_tr1",(Sigma_trcm*f_cmE+F_HE*Sigma_trHe+f_moderE*Sigma_trH2O)/(f_1))
        eta=add_in_table("eta",(Sigma_f5*nu_f5+Sigma_f9*nu_f9)/Sigma_a_0)
    if VAR==1 or VAR==2 or VAR==44:
        if VAR==1 or VAR==44:
            mu_0=add_in_table("mu_0",1+0.27*d_f*(N_5+N_8)*1e-24*100,QU=6)
        
            P=add_in_table("P",0.154*dens_H2O*f_moderE/(N_5+N_8)/1e-24/F_fuel)
            epsilon=add_in_table("epsilon",(1.34+P*mu_0)/(1+P))
        
            S_s=add_in_table("S_s",sympy.pi*d_f*H*1e4)
            M_8=add_in_table("M_8",N_8*sympy.pi*d_f**2/4*1e6*H*A_8/N_a)
            PP_t=add_in_table("П_т",0.77+0.0135*sympy.sqrt(t_U))
            I_ef=add_in_table("I_ef",4.9+20.7*sympy.sqrt(S_s/M_8)*PP_t)
            varphi=add_in_table("varphi",sympy.exp(-F_fuel/f_1/Sigma_ksi_s_1*N_8*1e-24*I_ef))
        if VAR==2:
            epsilon=add_in_table("epsilon",epsilon_start,show=False)
            varphi=add_in_table("varphi",varphi_start,show=False)
        A=add_in_table("A",0.2707*(d_f*100/2*Sigma_tr0)+0.1796*(d_f*100/2*Sigma_tr0)**2)
        a_forQ=add_in_table("a_forQ",0.067*(d_f*100/2*Sigma_tr0)-0.0037*(d_f*100/2*Sigma_tr0)**2)
        b=add_in_table("b",(0.046*(d_f*100/2*Sigma_tr0)-0.02*(d_f*100/2*Sigma_tr0)**2)-(0.046*(d_f*100/2*Sigma_tr0)-0.02*(d_f*100/2*Sigma_tr0)**2)*sympy.Heaviside((d_f*100*Sigma_tr0/2),H0=0)+0.027*sympy.Heaviside((d_f*100*Sigma_tr0/2),H0=0))
        Q_0=add_in_table("Q_0",1+Sigma_a_0/Sigma_tr0*A*(1+a_forQ*Sigma_s_0/Sigma_tr0+b*(Sigma_s_0/Sigma_tr0)**2))#a or s
        q_0=add_in_table("q_0",Q_0*Sigma_a_1/Sigma_a_0*f_1/F_fuel)
        L_1=add_in_table("L_1",sympy.sqrt(1/3/Sigma_a_1/Sigma_tr1))
        psi=add_in_table("psi",2*R_E/d_f)
        q_1=add_in_table("q_1",0.5*(R_E*100/L_1)**2*(psi**2/(psi**2-1)*sympy.log(psi)-3/4+1/4/psi))
        q=add_in_table("q",q_0+q_1)
        theta=add_in_table("theta",1/(1+q))
            
        K_inf=add_in_table("K_inf",theta*varphi*epsilon*eta)
        
        tau_H2O=add_in_table("tau_H2O",31.4)
        tau_t=add_in_table("tau_t",tau_H2O*(1+1/W+f_cmE/f_moderE)**2/(dens_H2O+0.82/W+0.4*f_cmE/f_moderE)**2)
        L_0=add_in_table("L_0",sympy.sqrt(1/3/Sigma_a_0/Sigma_tr0))
        L2=add_in_table("L^2",L_0**2*theta+(1-theta)*L_1**2)
        M2=add_in_table("M^2",L2+tau_t)
        delta_eff=add_in_table("delta_eff",7.2+0.1*(M2-40))
        H_eff=add_in_table("H_eff",H*100+2*delta_eff)
        R_eff=add_in_table("R_eff",D/2*100+delta_eff)
        B2=add_in_table("B^2",(sympy.pi/H_eff)**2+(2.405/R_eff)**2)
        P_mod=add_in_table("P_mod",sympy.exp(-B2*tau_t))
        P_dif=add_in_table("P_dif",1/(1+L2*B2))
        K_eff=add_in_table("K_eff",K_inf*P_mod*P_dif,QU=6)
    return graph
    line(3)
    



## Расчет эффективной температуры нейтронного газа

In [6]:
xi=0.03
g_fc=main(xi=xi)
def mainfh(tw=None):
    def it(tt):
        Showing=False
        transfer_data=False
        graph=graph_copy.copy()
        tt2=main(xi=xi,Temp=True,cold=False,VAR=-1,TTT=tt)
        if abs(tt-tt2)>1e-2:
            return it(tt2)
        else:
            return tt2
    Showing=False
    transfer_data=False
    graph=graph_copy.copy()
    tt=it(main(xi=xi,Temp=True,cold=False,VAR=-1))
    if tw==None:
        return main(xi=xi,TTT=tt,Temp=True,cold=False)
    else:
        return tt
Showing=False
transfer_data=False
tt=mainfh(tw=True)
Showing=True
transfer_data=True
graph=graph_copy.copy()
g_fh=main(xi=xi,TTT=tt,Temp=True,cold=False)


Eq(X, 0.03)

Eq(σ_s5, 1.38e-23)

Eq(σ_s8, 8.9e-24)

Eq(σ_sZr, 6.4e-24)

Eq(σ_sO2, 3.76e-24)

...

Eq(Sigma_s_1, 2.11)

Eq(Σξ_s_1, (F_HE*Σξ_sHe + f_cmE*Σξ_scm + f_moderE*Σξ_sH2O)/f_1)

Eq(Σξ_s_1, 0.77)

Eq(T_H, t_aver*(1 + (1.7*F_fuel*Sigma_a_0 + 1.7*Sigma_a_1*f_1)/(f_1*Σξ_s_1)))

Eq(T_H, 761.0)

## Расчет холодного//горячего реактора на начало кампании

In [7]:
def main2(gr,cold=True,xi=xi):
    TTT=gr[list(gr.keys())[-1]]
    graph=main(xi=xi,TTT=TTT,Temp=True,VAR=1,cold=cold)
    graph3=graph.copy()
    graph=graph_copy.copy()
    return graph3
g_sc=main2(g_fc)
g_sh=main2(g_fh,cold=False)

Eq(X, 0.03)

Eq(σ_s5, 1.38e-23)

Eq(σ_s8, 8.9e-24)

Eq(σ_sZr, 6.4e-24)

Eq(σ_sO2, 3.76e-24)

...

Eq(P_mod, 0.977)

Eq(P_dif, 1/(B^2*L^2 + 1))

Eq(P_dif, 0.999)

Eq(K_eff, K_inf*P_dif*P_mod)

Eq(K_eff, 1.0649)

## Расчет горячего реактора на конец кампании

In [8]:
TTT=g_fh[list(g_fh.keys())[-1]]
print(list(g_sh.keys())[-28])
epsilon=g_sh[list(g_sh.keys())[-29]]
varphi=g_sh[list(g_sh.keys())[-24]]
arrr=[epsilon,varphi]
graph=graph_copy.copy()
gr_end=main(xi=xi,TTT=TTT,Temp=True,VAR=2,cold=False,mas=arrr)

S_s


Eq(X, 0.03)

Eq(σ_s5, 1.38e-23)

Eq(σ_s8, 8.9e-24)

Eq(σ_sZr, 6.4e-24)

...

Eq(P_mod, 0.977)

Eq(P_dif, 1/(B^2*L^2 + 1))

Eq(P_dif, 0.999)

Eq(K_eff, K_inf*P_dif*P_mod)

Eq(K_eff, 0.913956)

# Таблица

In [9]:
from openpyxl import load_workbook
def write_in_excel(sheet_name,arr):
    global data
    book = load_workbook('demo.xlsx')
    writer = pd.ExcelWriter('demo.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    data.to_excel(writer,sheet_name=sheet_name,columns=arr);
    writer.save()
write_in_excel('inv',["Name","Num"])
print(data)

      Name                   Num
0      Q_p                  1399
0      q_v                    83
0      T_p                  7000
0      P_1                    16
0   t_aver                   580
..     ...                   ...
0    R_eff      148.823814037057
0      B^2  0.000372549644841008
0    P_mod     0.976859708953787
0    P_dif     0.998544764120820
0    K_eff     0.913955695920291

[585 rows x 2 columns]


## Подбор оптимального обогащения реактора

In [10]:
Showing=False
graph=graph_copy.copy()
def find_X(x,res=False,plot=False,VAR=None):
    try:
        xi=float(x[0])
        g_fh=mainfh()
        g_sh=main2(g_fh,xi=xi,cold=False)
        TTT=g_fh[list(g_fh.keys())[-1]]
        epsilon=g_sh[list(g_sh.keys())[-29]]
        varphi=g_sh[list(g_sh.keys())[-24]]
        arrr=[epsilon,varphi]
        Showing=False
        gr_end=main(xi=xi,TTT=TTT,Temp=True,VAR=2,cold=False,mas=arrr)
        K_eff=gr_end[list(gr_end.keys())[-1]]
        if VAR==44:
            Showing=False
            K_1=g_sh[list(g_sh.keys())[-1]]
            g_sc=main2(g_fh,xi=xi,cold=True)
            K_2=g_sc[list(g_sc.keys())[-1]]
            arrr.append(abs(K_2-K_1))
            Showing=True
            gr_end=main(xi=xi,TTT=TTT,Temp=True,VAR=44,cold=False,mas=arrr)
        if res==True:
            K=sympy.symbols("K_eff")
            disp.display(sympy.Eq(K,sympy.N(K_eff,3)))
        if plot==False:
            return abs(K_eff-1)
        else:
            return K_eff
    except Exception:
        return 1e10
import scipy.optimize as opt
bounds = [[0.02,0.8]]
x0=[0.03]
result = opt.minimize(find_X,x0, bounds=bounds)
ans=result.x[0]
X=sympy.symbols("X")
disp.display(sympy.Eq(X,sympy.N(ans*100,5)))
a=find_X([ans],res=True)

Eq(X, 3.7615)

Eq(K_eff, 1.0)